# Louvain Community Detection


In this notebook, we will use cuGraph to identify the cluster in a test graph using the Louvain algorithm  

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 03/08/2019

RAPIDS Versions: 0.6.0

Test Hardware
* GP100 32G, CUDA 9,2



## Introduction

The Louvain method of community detection is a greedy heirarical clsutering algorithm which seeks to optimize Modularity as it progresses. Louvain starts with each vertex in its own clusters and iteratively merges groups using graph contraction.  

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Louvain_Modularity

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score


To compute the Louvain cluster in cuGraph use: 
**nvLouvain(G)**
* __G__: A cugraph.Graph object

Returns:

* tupal __lovain dataframe__ and __modularity__


* __louvain__: cudf.DataFrame with two names columns:
    * louvain["vertex"]: The vertex id.
    * louvain["partition"]: The assigned partition.
    
* __modularity__ : the overall modularity of the graph

All vertices with the same partition ID are in the same cluster


## cuGraph 0.6 Notice 
cuGraph version 0.6 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations    
    

## References
* https://research.nvidia.com/publication/2017-11_Parallel-Jaccard-and


* Blondel, V. D., Guillaume, J.-L., Lambiotte, R., and Lefebvre, E. Fast unfolding of communities in large networks. Journal of statistical mechanics: theory and experiment 2008, 10 (2008), P10008.


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from collections import OrderedDict

## Read data using cuDF

In [ ]:
# Test file    
datafile='../data/networks/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [ ]:
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [ ]:
# The algorithm requires that there are vertex weights
gdf["data"] = 1.0

In [ ]:
gdf.dtypes

In [ ]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"], gdf["data"])

In [ ]:
# Call Louvain on the graph
df, mod = cugraph.nvLouvain(G) 

In [ ]:
# Print the modularity score
print('Modularity was {}'.format(mod))
print()

In [ ]:
df.dtypes

In [ ]:
# How many partitions where found
part_ids = df["partition"].unique()

In [ ]:
print("Found " + str(len(part_ids)) + " partition")

In [ ]:
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'][i] == p):
            part.append(df['vertex'][i])
    print("Partition " + str(p) + ":")
    print(part)


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___